In [1]:
import pandas as pd
import duckdb
from pathlib import Path

In [2]:
%load_ext watermark
%watermark -v -p duckdb,pandas

Python implementation: CPython
Python version       : 3.12.9
IPython version      : 9.9.0

duckdb: 1.4.3
pandas: 2.3.3



### Read Data Comparison

In [3]:
%%time
dfp = pd.read_parquet("artists.parquet")

CPU times: total: 13 s
Wall time: 10.7 s


In [4]:
%%time
dfp = pd.read_parquet("artists.parquet", engine='fastparquet')

CPU times: total: 4 s
Wall time: 4.13 s


In [5]:
%%time
dfdb = duckdb.read_parquet("artists.parquet")

CPU times: total: 0 ns
Wall time: 3.5 ms


In [6]:
%%time
dfd = duckdb.sql("""
  SELECT *  FROM 'artists.parquet'
""").df()

CPU times: total: 6.47 s
Wall time: 3.67 s


In [7]:
con = duckdb.connect()
con.execute(
    """
  CREATE OR REPLACE VIEW artists AS
  SELECT * FROM read_parquet('artists.parquet');
"""
)

In [8]:
%%time


dfd = con.execute(
    """
  SELECT * FROM read_parquet('artists.parquet');
"""
).fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 6.31 s
Wall time: 6.09 s


### Load and Filter

In [9]:
%%time
dfp = pd.read_parquet("artists.parquet").query("fetched_at >= 1743033600000 and followers_total > 0")

CPU times: total: 14.7 s
Wall time: 12.4 s


In [10]:
%%time
artists_df  = pd.read_parquet("artists.parquet")
artists_filter = artists_df [(artists_df ["fetched_at"] >= 1743033600000) & (artists_df ["followers_total"] > 0)]

CPU times: total: 13.7 s
Wall time: 11.2 s


In [11]:
df = duckdb.sql(
    """
  SELECT rowid, id, fetched_at, name, followers_total, popularity
  FROM 'artists.parquet'
  WHERE fetched_at >= 1743033600000
    AND followers_total > 0
"""
).df()

In [12]:
%%time

filtered = con.sql("""
  SELECT rowid, id, fetched_at, name, followers_total, popularity
  FROM artists
  WHERE fetched_at = 1743033600000
    AND followers_total > 0
""").df()


CPU times: total: 2.36 s
Wall time: 1.23 s


### Latest Fetched

In [13]:
%%time
latest = con.sql(
    """
  SELECT rowid, id, fetched_at, name, followers_total, popularity
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id ORDER BY fetched_at DESC, rowid DESC) AS rn
    FROM artists
  )
  WHERE rn = 1
"""
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 18.7 s
Wall time: 6.21 s


In [14]:
%%time
latest_df = artists_df.sort_values(
    ["id", "fetched_at", "rowid"], ascending=[True, False, False]
).drop_duplicates("id", keep="first")[
    ["rowid", "id", "fetched_at", "name", "followers_total", "popularity"]
]

CPU times: total: 1min 10s
Wall time: 1min 12s


### Daily Aggregation

In [15]:
%%time
daily = con.sql("""
  SELECT
    DATE(TIMESTAMP 'epoch' + fetched_at/1000 * INTERVAL '1 second') AS day,
    COUNT(*) AS rows,
    AVG(followers_total) AS avg_followers,
    APPROX_QUANTILE(followers_total, 0.50) AS p50_followers,
    APPROX_QUANTILE(followers_total, 0.90) AS p90_followers,
    AVG(popularity) AS avg_popularity
  FROM artists
  GROUP BY day
  ORDER BY day
""").df()


CPU times: total: 3.72 s
Wall time: 556 ms


In [16]:
tmp = artists_df.copy()
tmp["day"] = pd.to_datetime(tmp["fetched_at"], unit="ms").dt.date

In [17]:
%%time
daily_df = (tmp.groupby("day")
    .agg(rows=("id","size"),
         avg_followers=("followers_total","mean"),
         p50_followers=("followers_total", lambda s: s.quantile(0.50)),
         p90_followers=("followers_total", lambda s: s.quantile(0.90)),
         avg_popularity=("popularity","mean"))
    .reset_index()
    .sort_values("day")
)


CPU times: total: 1.72 s
Wall time: 1.71 s


### Top 100 most followed artists

In [18]:
%%time
top100 = con.sql("""
  WITH latest AS (
    SELECT *
    FROM (
      SELECT *,
             ROW_NUMBER() OVER (PARTITION BY id ORDER BY fetched_at DESC, rowid DESC) AS rn
      FROM artists
    )
    WHERE rn = 1
  )
  SELECT id, name, followers_total, popularity, fetched_at
  FROM latest
  ORDER BY followers_total DESC
  LIMIT 100
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 15.6 s
Wall time: 3.78 s


In [19]:
%%time
top100_df = (latest_df
    .sort_values("followers_total", ascending=False)
    .head(100)[["id","name","followers_total","popularity","fetched_at"]]
)


CPU times: total: 3.45 s
Wall time: 3.59 s


### DQ Checks

In [20]:
%%time
dq = con.sql("""
  SELECT
    COUNT(*) AS rows,
    COUNT(DISTINCT id) AS distinct_ids,
    SUM(CASE WHEN name IS NULL OR name = '' THEN 1 ELSE 0 END) AS missing_name,
    SUM(CASE WHEN followers_total < 0 THEN 1 ELSE 0 END) AS negative_followers,
    SUM(CASE WHEN popularity < 0 OR popularity > 100 THEN 1 ELSE 0 END) AS popularity_out_of_range
  FROM artists
""").df()


CPU times: total: 5.59 s
Wall time: 1.1 s


In [21]:
%%time
dq_df = {
  "rows": int(len(artists_df)),
  "distinct_ids": int(artists_df["id"].nunique()),
  "missing_name": int(artists_df["name"].isna().sum() + (artists_df["name"] == "").sum()),
  "negative_followers": int((artists_df["followers_total"] < 0).sum()),
  "popularity_out_of_range": int(((artists_df["popularity"] < 0) | (artists_df["popularity"] > 100)).sum()),
}

CPU times: total: 13 s
Wall time: 13.4 s


### Writing files

In [22]:
%%time
con.execute("""
  COPY artists
  TO 'artists_out.parquet'
  (FORMAT PARQUET, CODEC 'ZSTD');
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 16.3 s
Wall time: 2.49 s


In [23]:
%%time
# alternatively
dfdb.write_parquet("artists_out_duckdb.parquet", compression="zstd")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 17 s
Wall time: 2.23 s


In [24]:
%%time
con.execute("""
  COPY artists
  TO 'artists_out_lz4.parquet'
  (FORMAT PARQUET, CODEC 'lz4');
""")


CPU times: total: 11.8 s
Wall time: 1.52 s


In [25]:
%%time
con.execute("""
  COPY artists
  TO 'artists_out_brotli.parquet'
  (FORMAT PARQUET, CODEC 'brotli');
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 27min 36s
Wall time: 3min


In [26]:
%%time
con.execute("""
  COPY artists
  TO 'artists_out_gzip.parquet'
  (FORMAT PARQUET, CODEC 'gzip');
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 1min
Wall time: 6.57 s


In [27]:
%%time
con.execute("""
  COPY artists
  TO 'artists.csv'
  (FORMAT CSV, HEADER, DELIMITER ',');
""")

CPU times: total: 9.92 s
Wall time: 1.25 s


In [28]:
%%time
artists_df.to_parquet(
    "artists_out_pandas.parquet", engine="pyarrow", compression="zstd", index=False
)

CPU times: total: 7.56 s
Wall time: 7.7 s


In [29]:
%%time
artists_df.to_parquet(
    "artists_out_pandas_fp.parquet",
    engine="fastparquet",
    compression="zstd",
    index=False,
)

CPU times: total: 11.5 s
Wall time: 11.7 s


In [30]:
%%time
res = {}
for p in Path.cwd().iterdir():
    if ".ipynb" not in str(p):
        res[str(p).split("\\")[-1]] = round(p.stat().st_size / 1024**2, 4)

CPU times: total: 0 ns
Wall time: 0 ns


In [31]:
pd.DataFrame([res]).T.rename(columns={0: "Size (MB)"})

,Size (MB)
artists.csv,925.5315
artists.parquet,615.9072
artists_out.parquet,410.0665
artists_out_brotli.parquet,364.7962
artists_out_duckdb.parquet,410.0665
artists_out_gzip.parquet,421.6951
artists_out_lz4.parquet,635.1817
artists_out_pandas.parquet,435.7401
artists_out_pandas_fp.parquet,409.5974
readme.md,0.0020
